# 모델 매개변수 최적화 하기
- 모델 학습은 반복적인 과정을 거침.
- 출력 추측, 정답과 추측 사이의 loss를 계산하고, 매개변수에 대한 오류의 도함수(derivate)를 수집한 뒤, 경사하강법을 사용해 파라미터를 optimizer한다.

In [17]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
	root = "data",
	train = True,
	download = True,
	transform = ToTensor()
)

test_data = datasets.FashionMNIST(
	root = 'data',
	train = False,
	download = True,
	transform = ToTensor()
)


In [18]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


In [19]:
class NeuralNetwork(nn.Module):
	def __init__(self):
		super().__init__()
		self.flatten = nn.Flatten()
		self.linear_relu_stack = nn.Sequential(
			nn.Linear(28*28, 512),
			nn.ReLU(),
			nn.Linear(512, 512),
			nn.ReLU(),
			nn.Linear(512, 10)
		)
	def forward(self, x):
		x = self.flatten(x)
		logits = self.linear_relu_stack(x)
		return logits

model = NeuralNetwork()


## 하이퍼파라미터(Hyperparameter)
모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수. 서로 다른 하이퍼파라미터 값은 모델학습과 수렴율(convergence rate)에 영향을 미칠 수 있음. 
- 에폭(Epoch) 수 - 데이터셋을 반복하는 횟수
- 배치 크기(Batch_size) - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터의 샘플 수
- 학습률(Learning_rate) - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율. 값이 작을 수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있음.

In [20]:
learning_rate = 1e-3
batch_size = 64
epochs = 5


## Optimizer Loop(최적화 단계)
하이퍼파라미터를 설정한 뒤에는 모델을 최적화 단계를 통해 학습&최적화가 가능. 각 반복을 epoch이라 부른다.
- train loop - 학습용 데이터셋을 반복해서 최적의 매개변수로 수렴
- validation/test loop - 모델 성능이 개선되고 있는지를 확인하기 위해 테스트 데이터셋을 반복

### Loss function
학습용 데이터를 제공하면 학습되지 않은 신경망은 정답이 아닐 확률이 높다. loss function은 획득한 결과와 실제 값 사이의 틀린 정도(degree of dissimilarity)를 측정해 이 값을 최소화한다.
- 데이터 샘플을 입력으로 계산하고, 예측과 정답을 비교해서 손실을 계산한다.

In [21]:
loss_fn = nn.CrossEntropyLoss()


### Optimizer
각 학습단계에서 오류를 줄이기 위해 매개변수를 조절하는 과정. 여기서는 SGD(Stochastic Gradient Descent)
- 모든 최적화 절차는 `optimizer`객체에 캡슐화됨. 
- 학습하려는 모델의 매개변수와 학습률 하이퍼파라미터를 등록해 옵티마이저를 초기화 한다.

In [22]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


#### loop 최적화 단계는 3개
- `optimizer.zero_grad()`를 호출해서 모델 매개변수의 변화도를 재설정한다. 기본적으로 변화도는 add up이기 때문에 중복 계산을 막기 위해 반복마다 0으로 설정함
- `loss.backward()`를 호출해서 예측손실(prediction loss)을 역전파한다. 
- 변화도를 계산한 뒤에는 `optimizer.step()`을 호출해서 역전파 단계에서 수집된 변화도로 매개변수를 조정

In [27]:
def train_loop(dataloader, model, loss_fn, optimizer):
	size = len(dataloader.dataset)
	model.train()

	for batch, (X, y) in enumerate(dataloader):
		pred = model(X)
		loss = loss_fn(pred, y)

		loss.backward()
		optimizer.step()
		optimizer.zero_grad()

		if batch % 100 == 0:
			loss, current = loss.item(), batch * batch_size + len(X)
			print(f"loss: {loss:>7f} [{current:>5d} / {size::>5d}]")

def test_loop(dataloader, model, loss_fn):
	size = len(dataloader.dataset)
	model.eval()
	num_batches = len(dataloader)
	test_loss, correct = 0, 0

	with torch.no_grad():
		for X, y in dataloader:
			pred = model(X)
			test_loss = loss_fn(pred, y).item()
			correct += (pred.argmax(1) == y).type(torch.float).sum().item()

	test_loss /= num_batches
	correct /= size
	print(f"Test: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")


`.item()`을 하지 않을 경우 test_loss가 값이아닌 텐서가 저장되서 `test_loss /= num_batches`와 같은 연산에서 경고가 발생할 수 있음. 

In [31]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
	print(f"Epoch {t+1}\n-------------")
	train_loop(train_dataloader, model, loss_fn, optimizer)
	test_loop(test_dataloader, model, loss_fn)
print("Done")


Epoch 1
-------------
loss: 0.447659 [   64 / 60000]
loss: 0.563775 [ 6464 / 60000]
loss: 0.370152 [12864 / 60000]
loss: 0.600224 [19264 / 60000]
loss: 0.529290 [25664 / 60000]
loss: 0.531633 [32064 / 60000]
loss: 0.528099 [38464 / 60000]
loss: 0.670642 [44864 / 60000]
loss: 0.623863 [51264 / 60000]
loss: 0.508385 [57664 / 60000]
Test: 
 Accuracy: 81.0%, Avg loss: 0.002020

Epoch 2
-------------
loss: 0.441464 [   64 / 60000]
loss: 0.558522 [ 6464 / 60000]
loss: 0.366052 [12864 / 60000]
loss: 0.594770 [19264 / 60000]
loss: 0.524166 [25664 / 60000]
loss: 0.527489 [32064 / 60000]
loss: 0.523316 [38464 / 60000]
loss: 0.671091 [44864 / 60000]
loss: 0.622273 [51264 / 60000]
loss: 0.501736 [57664 / 60000]
Test: 
 Accuracy: 81.2%, Avg loss: 0.002003

Epoch 3
-------------
loss: 0.435487 [   64 / 60000]
loss: 0.553607 [ 6464 / 60000]
loss: 0.362149 [12864 / 60000]
loss: 0.589512 [19264 / 60000]
loss: 0.519218 [25664 / 60000]
loss: 0.523395 [32064 / 60000]
loss: 0.518847 [38464 / 60000]
loss: 0